In [3]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

# Read model (model of Escherichia coli)
model = read_sbml_model('iML1515.xml.gz')

In [1]:
model.metabolites.get_by_id('34hpp_c')
#starting point
#add 5 reactions: 
#homogentisate 
#6-Geranylgeranyl-2-methylbenzene-1,4-diol 
#6-Geranylgeranyl-2,3-dimethylbenzene-1,4-diol 
#gamma-Tocotrienol
#alpha-Tocotrienol

NameError: name 'model' is not defined

In [5]:
# Co-factor production
# Reaction CF 1: frdp_c + ipdp_c ⇌ ggdp_c + h_c + ppi_c
new_reaction_CF1 = Reaction('GGPS') # Geranylgeranyl diphosphate synthase
geranylgeranyldiphosphate = Metabolite(id='ggdp_c', name='Geranylgeranyldiphosphate', compartment='c')
new_reaction_CF1.add_metabolites({model.metabolites.frdp_c: -1, # Farnesyl diphosphate
                               model.metabolites.ipdp_c: -1, # Isopentenyl diphosphat
                               geranylgeranyldiphosphate: 1, # Geranylgeranyl diphosphate
                               model.metabolites.ppi_c: 1, # diphosphate
                               model.metabolites.h_c: 1, # H+
                              })
model.add_reactions([new_reaction_CF1])

In [6]:
#model.metabolites.ggdp_c

Metabolite identifier,ggdp_c
Name,Geranylgeranyldiphosphate
Memory address,0x7fe77bef73d0
Formula,None
Compartment,c
In 1 reaction(s),GGPS


In [7]:
# Reaction 1: 3-(4-Hydroxyphenyl)pyruvate + O2 O2 --> CO2 CO2 + Homogentisate
new_reaction1 = Reaction('34HPPOR') # 4 Hydroxyphenylpyruvateoxygen oxidoreductase
homogentisate = Metabolite(id='hgentis_c', compartment='c')
new_reaction1.add_metabolites({model.metabolites.get_by_id('34hpp_c'): -1, # 3-(4-Hydroxyphenyl)pyruvate
                               model.metabolites.o2_c: -1, #O2
                               homogentisate: 1, # Homogentisate
                               model.metabolites.co2_c: 1, # CO2
                              })
model.add_reactions([new_reaction1])

In [8]:
# Reaction 2: ggdp_c + hgentis_c ⇌ co2_c + ppi_c + 6gg2mb14d_c
new_reaction2 = Reaction('GGTF') # Geranylgeranyltransferase
geranylgeranylmethylbenzenediol = Metabolite(id='6gg2mb14d_c', name='6-Geranylgeranyl-2-methylbenzene-1,4-diol', compartment='c')
new_reaction2.add_metabolites({model.metabolites.hgentis_c: -1, # Homogentisate
                               model.metabolites.ggdp_c: -1, # Geranylgeranyl diphosphate
                               geranylgeranylmethylbenzenediol: 1, # 6-Geranylgeranyl-2-methylbenzene-1,4-diol
                               model.metabolites.ppi_c: 1, # diphosphate
                               model.metabolites.co2_c: 1, # CO2
                              })
model.add_reactions([new_reaction2])

In [9]:
print(new_reaction2.build_reaction_string())

ggdp_c + hgentis_c --> 6gg2mb14d_c + co2_c + ppi_c


In [10]:
# Reaction 3: amet_c + 6gg2mb14d_c ⇌ ahcys_c + 6gg23dmb14d_c
new_reaction3 = Reaction('GGDMTF') # 6-geranylgeranyl-2-methylbenzene-1,4-diol C3-methyltransferase
dimethylbenzene = Metabolite(id='6gg23dmb14d_c', name='6-geranylgeranyl-2-methylbenzene-1,4-diol', compartment='c')
new_reaction3.add_metabolites({model.metabolites.get_by_id('6gg2mb14d_c'): -1, # 6-Geranylgeranyl-2-methylbenzene-1,4-diol
                               model.metabolites.amet_c: -1, # S-Adenosyl-L-methionine
                               dimethylbenzene: 1, # 6-geranylgeranyl-2-methylbenzene-1,4-diol
                               model.metabolites.ahcys_c: 1, # S-Adenosyl-L-homocysteine
                              })
model.add_reactions([new_reaction3])

In [11]:
print(new_reaction3.build_reaction_string())

6gg2mb14d_c + amet_c --> 6gg23dmb14d_c + ahcys_c


In [12]:
# Reaction 4: 6gg2mb14d_c ⇌ gtocotri_c
new_reaction4 = Reaction('TOCOTRI1') # 	Gamma-tocotrienol lyase (decyclizing)
gammatocotrienol = Metabolite(id='gtocotri_c', compartment='c')
new_reaction4.add_metabolites({model.metabolites.get_by_id('6gg2mb14d_c'): -1, # 6-geranylgeranyl-2-methylbenzene-1,4-diol
                               gammatocotrienol: 1, # Gamma-Tocotrienol
                              })
model.add_reactions([new_reaction4])

In [13]:
print(new_reaction4.build_reaction_string())

6gg2mb14d_c --> gtocotri_c


In [14]:
# Reaction 5: amet_c + gtocotri_c ⇌ ahcys_c  + avite_GG_c
new_reaction5 = Reaction('TOCOTRI_GG_1') #  S-adenosyl-L-methionine:gamma-tocotrienol 5-O-methyltransferase
alphatocotrienol = Metabolite(id='avite_GG_c', compartment='c')
new_reaction5.add_metabolites({model.metabolites.gtocotri_c: -1, # Gamma-Tocotrienol
                               model.metabolites.amet_c: -1, # S-Adenosyl-L-methionine
                               alphatocotrienol: 1, # Alpha-Tocotrienol
                               model.metabolites.ahcys_c: 1, # S-Adenosyl-L-homocysteine
                              })
model.add_reactions([new_reaction5])

In [15]:
print(new_reaction5.build_reaction_string())

amet_c + gtocotri_c --> ahcys_c + avite_GG_c


In [16]:
#production of vitamin E (avite_GG_c)
model.add_boundary(model.metabolites.avite_GG_c, type='demand')

with model:
    model.objective = model.reactions.TOCOTRI_GG_1
    lets_produce_avite1 = model.optimize().objective_value

In [17]:
#production rate of vitamin E
lets_produce_avite1

1.3418368043700932

In [18]:
from cobra.io import write_sbml_model

In [20]:
write_sbml_model(model,"modelavite.xml")